In [57]:
import polars as pl

In [76]:
seq_names = ["a_c_60_NNNNNNNN_mx3", "a_c_65_pblibTACTGGCT_mx3", "a_c_30_BNNNNNNN_mx1"]

In [77]:
df = pl.DataFrame({"raw_name":seq_names})

In [78]:
df = df.with_columns(split_name=pl.col("raw_name").str.split("_"))

In [79]:
df

raw_name,split_name
str,list[str]
"""a_c_60_NNNNNNNN_mx3""","[""a"", ""c"", … ""mx3""]"
"""a_c_65_pblibTACTGGCT_mx3""","[""a"", ""c"", … ""mx3""]"
"""a_c_30_BNNNNNNN_mx1""","[""a"", ""c"", … ""mx1""]"


In [84]:
df = df.with_columns(
    lineage=pl.col("split_name").list.get(-1),
    timepoint=pl.col("split_name").list.get(2),
    umi=pl.col("split_name").list.get(-2).str.slice(5,8),
)

In [85]:
df

raw_name,split_name,lineage,timepoint,umi
str,list[str],str,str,str
"""a_c_60_NNNNNNNN_mx3""","[""a"", ""c"", … ""mx3""]","""mx3""","""60""","""NNN"""
"""a_c_65_pblibTACTGGCT_mx3""","[""a"", ""c"", … ""mx3""]","""mx3""","""65""","""TACTGGCT"""
"""a_c_30_BNNNNNNN_mx1""","[""a"", ""c"", … ""mx1""]","""mx1""","""30""","""NNN"""


In [46]:
all_timepoints = sorted(df["timepoint"].unique().to_list())
all_lineages = sorted(df["lineage"].unique().to_list())

In [47]:
df = df.with_columns(
    lineage_num=pl.col("lineage").map_elements(
        lambda x: all_lineages.index(x) + 1, return_dtype=int
    ),
    timepoint_num=pl.col("timepoint").map_elements(
        lambda x: all_timepoints.index(x) + 1, return_dtype=int
    ),
)

In [52]:
df = df.with_columns(new_names=pl.col("umi")+"_TP"+pl.col("lineage_num").cast(str))

In [55]:
df.select(pl.col("new_names", "lineage_num", "lineage"))

new_names,lineage_num,lineage
str,i64,str
"""NNNNNNNN_TP2""",2,"""mx3"""
"""NNNNNANN_TP2""",2,"""mx3"""
"""BNNNNNNN_TP1""",1,"""mx1"""


ModuleNotFoundError: No module named 'numpy'

In [100]:
lineages = [
    "lineage_a",
    "lineage_b",
    "a_this_should_be1",
    "lineage_b",
    "lineage_b",
    "z_this_should_be_last",
    "a_this_should_be1",
    "a_this_should_be1",
    "a_this_should_be1",
    "lineage_a",
]
unique_lineages = sorted(list(set(lineages)))
print(unique_lineages)

for l in lineages:
    print(" | ".join([l, "index:", str(unique_lineages.index(l) + 1)]))
    


['a_this_should_be1', 'lineage_a', 'lineage_b', 'z_this_should_be_last']
lineage_a | index: | 2
lineage_b | index: | 3
a_this_should_be1 | index: | 1
lineage_b | index: | 3
lineage_b | index: | 3
z_this_should_be_last | index: | 4
a_this_should_be1 | index: | 1
a_this_should_be1 | index: | 1
a_this_should_be1 | index: | 1
lineage_a | index: | 2


['a_this_should_be1', 'lineage_a', 'lineage_b', 'z_this_should_be_last']


[('lineage_a', 2),
 ('lineage_b', 3),
 ('a_this_should_be1', 1),
 ('lineage_b', 3),
 ('lineage_b', 3),
 ('z_this_should_be_last', 4),
 ('a_this_should_be1', 1),
 ('a_this_should_be1', 1),
 ('a_this_should_be1', 1),
 ('lineage_a', 2)]